# Search AiiDA Database for Isotherms

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job import JobCalculation

import ase.io
import sys
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile
from base64 import b64encode


import nglview

In [ ]:
############################   START OF PREPROCESSING   ###############################

In [ ]:
PREPROCESS_VERSION = 0.19

def preprocess_newbies():
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={
        'attributes._process_label': 'Isotherm',
        'or':[
               {'extras': {'!has_key': 'preprocess_version'}},
               {'extras.preprocess_version': {'<': PREPROCESS_VERSION}},
           ],
    })
    
    
    for m in qb.all(): # iterall() would interfere with set_extra()
        n = m[0]
        """
        if not n.is_sealed:
            print("Skipping underway workchain PK %d"%n.pk)
            continue
        """
        if 'obsolete' not in n.get_extras():
            n.set_extra('obsolete', False)
        try:
            preprocess_one(n)
            n.set_extra('preprocess_successful', True)
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Preprocessed PK %d"%n.pk)
        except Exception as e:
            n.set_extra('preprocess_successful', False)
            n.set_extra('preprocess_error', str(e))
            n.set_extra('preprocess_version', PREPROCESS_VERSION)
            print("Failed to preprocess PK %d: %s"%(n.pk, e))
    

In [ ]:
def preprocess_one(workcalc):
    
    def get_calc_by_label(workcalc, label):
        qb = QueryBuilder()
        qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
        qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
        if qb.count() != 1:
            raise(Exception("Could not find %s calculation."%label))
        calc = qb.first()[0]
        return calc
    def get_calc_by_label_multi(workcalc, label):
        qb = QueryBuilder()
        qb.append(WorkCalculation, filters={'uuid':workcalc.uuid})
        qb.append(JobCalculation, output_of=WorkCalculation, filters={'label':label})
        return [i[0] for i in qb.all()]

    # clean extras
    for key in workcalc.get_extras():
        workcalc.del_extra(key)
    
    
    # formula
    structure = workcalc.inp.structure
    ase_struct = structure.get_ase()
    formula = ase_struct.get_chemical_formula()
    workcalc.set_extra('formula', formula)
    workcalc.set_extra('structure_description', structure.description)
    
    # analysis of the geometry
    geom_properties = get_calc_by_label(workcalc, "run_geom_zeopp")
    workcalc.set_extra('density', geom_properties.out.pore_volume_volpo.get_attr('Density'))
    workcalc.set_extra('density_units', 'g/cm^3')
    workcalc.set_extra('pore_volume', geom_properties.out.pore_volume_volpo.get_attr('POAV_A^3'))
    workcalc.set_extra('pore_volume_units', 'A^3')
    workcalc.set_extra('unitcell_volume', geom_properties.out.pore_volume_volpo.get_attr('Unitcell_volume'))
    workcalc.set_extra('unitcell_volume_units', 'A^3')
    workcalc.set_extra('largest_free_sphere', geom_properties.out.free_sphere_res.get_attr('Largest_free_sphere'))
    workcalc.set_extra('largest_included_free_sphere', geom_properties.out.free_sphere_res.get_attr('Largest_included_free_sphere'))
    workcalc.set_extra('largest_included_sphere', geom_properties.out.free_sphere_res.get_attr('Largest_included_sphere'))
    workcalc.set_extra('accessible_surface_area', geom_properties.out.surface_area_sa.get_attr('ASA_m^2/g'))
    workcalc.set_extra('accessible_surface_area_units', 'm^2/g')
    workcalc.set_extra('not_accessible_surface_area', geom_properties.out.surface_area_sa.get_attr('NASA_m^2/g'))
    workcalc.set_extra('not_accessible_surface_area_units', 'm^2/g')
    workcalc.set_extra('channel_surface_area', geom_properties.out.surface_area_sa.get_attr('Channel_surface_area_A^2'))
    workcalc.set_extra('channel_surface_area_units', 'A^2')
    workcalc.set_extra('number_of_channels', geom_properties.out.surface_area_sa.get_attr('Number_of_channels'))
    workcalc.set_extra('number_of_pockets', geom_properties.out.surface_area_sa.get_attr('Number_of_pockets'))   

    # extract isotherm
    isotherm_list = []
    raspa_calcs = get_calc_by_label_multi(workcalc, "run_loading_raspa")
    for rc in raspa_calcs:
        isotherm_list.append(
            {
                'pressure':          rc.inp.parameters.get_attr('GeneralSettings')['ExternalPressure']/1e5,
                'loading':           rc.out.component_0.get_attr('loading_absolute_average'),
                'loading_units':     rc.out.component_0.get_attr('loading_absolute_units'),
            })
    workcalc.set_extra('isotherm', sorted(isotherm_list, key=lambda k: k['pressure']))
    
    # extract deliverable capacity
    l5_8bar = next((item['loading'] for item in isotherm_list if item['pressure'] == 5.8), None)
    l65bar = next((item['loading'] for item in isotherm_list if item['pressure'] == 65), None)
    luni = next((item['loading_units'] for item in isotherm_list if item['pressure'] == 65), None)
    workcalc.set_extra('deliverable_capacity', l65bar - l5_8bar)
    workcalc.set_extra('deliverable_capacity_units', luni)
    
    
    
    #thumbnail = render_thumbnail(ase_struct)
    #structure.set_extra('thumbnail', thumbnail)

In [ ]:
def render_thumbnail(atoms):
    tmp = NamedTemporaryFile()
    ase.io.write(tmp.name, atoms, format='png') # does not accept StringIO
    raw = open(tmp.name).read()
    tmp.close()
    return b64encode(raw)

In [ ]:
preprocess_newbies()